# Conceitos avançados - Output Parsers
  
Os analisadores de saída são responsáveis por pegar a saída de um LLM e transformá-la em um formato mais adequado. Isso é muito útil quando você está usando LLMs para gerar qualquer forma de dados estruturados.   

  Digamos que temos a seguinte review de um produto

  > "Este soprador de folhas é bastante incrível. Ele tem quatro configurações: sopro de vela, brisa suave, cidade ventosa e tornado. Chegou em dois dias, bem a tempo para o     presente de aniversário da minha esposa. Acho que minha esposa gostou tanto que ficou sem palavras. Até agora, fui o único a usá-lo, e tenho usado em todas as manhãs   alternadas para limpar as folhas do nosso gramado. É um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado, mas acho que vale a pena pelas características extras."

  E eu quero que o modelo  de linguagem processe esta Review para estrutura--la no seguinte formato: 

  ```python
  {
  "presente": true,
  "dias_entrega": 2,
  "percepcao_de_valor": ["um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado"]
}

## Forma tradicional - sem Output Parsers

In [ ]:
# importando libs

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from dotenv import load_dotenv

# carregar arquivo .env
load_dotenv()

review_cliente= 'Este soprador de folhas é bastante incrível. Ele tem quatro configurações: sopro de vela, brisa suave, cidade ventosa e tornado. ' \
'Chegou em dois dias, bem a tempo para o presente de aniversário da minha esposa. ' \
'Acho que minha esposa gostou tanto que ficou sem palavras. Até agora, fui o único a usá-lo, e tenho usado em todas as manhãs   ' \
'alternadas para limpar as folhas do nosso gramado. É um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado, ' \
'mas acho que vale a pena pelas características extras.'

review_template = ChatPromptTemplate.from_template( '''

para o texto a seguir, extraia as seguiintes informações: 
presente: o item foi comprado para alguém? True caso verdadeiro
e Falso caso falso ou nãi tenha a informação

dias_entrega: Quantos dias para a entrega chegar?
Se a resposta não for encontrada retorne -1

percepcao_de_valor: extraia qualquer frase sobre o valor ou preço do produto. Retorne como uma lista de python

Texto: {review}

Retorn a resposta em formato Json
'''
)

print(review_template.format_messages(review=review_cliente))

[HumanMessage(content='\n\npara o texto a seguir, extraia as seguiintes informações: \npresente: o item foi comprado para algume? True caso verdadeiro\ne Falso caso falso ou nãi tenha a informação\n\ndias_entrega: Quantos dias para a entrega chegar?\nSe a resposta não for encontrada retorne -1\n\npercepcao_de_valor: extraia qualquer frase sobre o valor ou preço do produto. Retorne como uma lista de python\n\nTexto: Este soprador de folhas é bastante incrível. Ele tem quatro configurações: sopro de vela, brisa suave, cidade ventosa e tornado. Chegou em dois dias, bem a tempo para o presente de aniversário da minha esposa. Acho que minha esposa gostou tanto que ficou sem palavras. Até agora, fui o único a usá-lo, e tenho usado em todas as manhãs   alternadas para limpar as folhas do nosso gramado. É um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado, mas acho que vale a pena pelas características extras.\n\nRetorn a resposta em formato Json\n', additional_kwa

In [6]:
chat = ChatGoogleGenerativeAI(model='gemini-1.5-flash')
resposta = chat.invoke(review_template.format_messages(review=review_cliente))

print(resposta.content)

```json
{
  "presente": true,
  "dias_entrega": 2,
  "percepcao_de_valor": [
    "É um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado, mas acho que vale a pena pelas características extras."
  ]
}
```


## Melhor forma e mais aconcelhada - Com Output Parsers

In [7]:
# import libs


from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser



In [9]:
# criando Schema

schema_presente = ResponseSchema(
    name='presente',
    type='bool',
    description='o item foi comprado para alguém? True caso verdadeiro e Falso caso falso ou nãi tenha a informação'
)

schema_entrega = ResponseSchema(
    name='dias_entrega',
    type='int',
    description='Quantos dias para a entrega chegar? Se a resposta não for encontrada retorne -1'
)

schema_valor = ResponseSchema(
    name='percepcao_de_valor',
    type='list',
    description='extraia qualquer frase sobre o valor ou preço do produto Retorne como uma lista de python'

)

response_schemas = [schema_presente,
                  schema_entrega,
                  schema_valor]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

schema_formatado = output_parser.get_format_instructions()

print(schema_formatado)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"presente": bool  // o item foi comprado para alguém? True caso verdadeiro e Falso caso falso ou nãi tenha a informação
	"dias_entrega": int  // Quantos dias para a entrega chegar? Se a resposta não for encontrada retorne -1
	"percepcao_de_valor": list  // extraia qualquer frase sobre o valor ou preço do produto Retorne como uma lista de python
}
```


## Fazendo exerciccio do jeito correto e indicado ( Com OutPut Parser)

In [11]:
review_cliente = 'Este soprador de folhas é bastante incrível. Ele tem quatro configurações: sopro de vela, brisa suave, cidade ventosa e tornado. ' \
'Chegou em dois dias, bem a tempo para o presente de aniversário da minha esposa. ' \
'Acho que minha esposa gostou tanto que ficou sem palavras. Até agora, fui o único a usá-lo, e tenho usado em todas as manhãs   ' \
'alternadas para limpar as folhas do nosso gramado. É um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado, ' \
'mas acho que vale a pena pelas características extras.'


In [20]:
# importando libs

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

from langchain.output_parsers import StructuredOutputParser, ResponseSchema



In [ ]:


#criando schemas

schema_presente = ResponseSchema(
    name='presente',
    type='bool',
    description='o item foi comprado para alguém? True caso verdadeiro  \
          e Falso caso falso ou nãi tenha a informação'    
)

schema_entrega= ResponseSchema(
    name='dias_entrega',
    type='int',
    description='Quantos dias para a entrega chegar?  \
Se a resposta não for encontrada retorne -1'   
)

schema_valor =ResponseSchema (
    name='percepcao_de_valor',
    type='list',
    description='extraia qualquer frase sobre o valor ou preço do produto. Retorne como uma lista de python'
)

response_schemas = [schema_presente,
                    schema_entrega,
                    schema_valor
    ]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

schema_formatado = output_parser.get_format_instructions()

In [34]:
# enviando para ia 

review_template = ChatPromptTemplate.from_template(
''' 

para o texto a seguir, extraia as seguintes informações: 
presente, dias_entrega e percepcao_de_valor

Texto: {review}

{schema}
''', partial_variables={'schema': schema_formatado})



In [31]:
## chamando chjat para retorno da resposta

llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash')

resposta = llm.invoke(review_template.format_messages(review=review_cliente))

print(resposta.content)

```json
{
  "presente": true,
  "dias_entrega": 2,
  "percepcao_de_valor": ["É um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado, mas acho que vale a pena pelas características extras."]
}
```


In [ ]:
# agora é possivel dividir a resposta, visto que agora a resposta é um dicionário

resposta_json = output_parser.parse(resposta.content)

resposta_json['percepcao_de_valor']

['É um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado, mas acho que vale a pena pelas características extras.']